In [19]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(f"Loaded API Key starts with: {OPENAI_API_KEY[:8]}")

Loaded API Key starts with: sk-proj-


In [20]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

In [21]:
model.invoke("Who won the world cup in 2022?")

AIMessage(content='The 2022 FIFA World Cup has not yet taken place. It is scheduled to be held in Qatar from November 21 to December 18, 2022.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 17, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BnXOODZ3xK61FD1DoB5R5pwCcRZuu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7714270f-b3b9-40de-8a48-7311bfdea923-0', usage_metadata={'input_tokens': 17, 'output_tokens': 34, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [22]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

'The Los Angeles Dodgers won the World Series during the COVID-19 pandemic, defeating the Tampa Bay Rays in six games in the 2020 World Series.'

In [23]:
import fitz

doc = fitz.open("Lionel-June25.pdf") 
text = ""

for i, page in enumerate(doc):
    text += page.get_text()

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_text(text) 

documents

['ESTIMATE\nHammer Time Home\nImprovement HIC#0668980\n281 Middle Tpke E\nManchester, CT 06040\n860hammertime@gmail.com\n+1 (860) 830-7982\nfacebook.com/860hammertime\nBill to\nSatish Kulkarni\n1006 Sandstone Drive\nSouth Windsor, CT 06074\nShip to\nSatish Kulkarni\n1006 Sandstone Drive\nSouth Windsor, CT 06074\nEstimate details\nEstimate no.: 721\nEstimate date: 06/09/2025\nExpiration date: 07/09/2025\n#\nDate\nProduct or service\nDescription\nAmount\n1.\nRepair treads and risers on the set of stairs\nleading to 2nd floor of residence.\nRemove drywall from under stairs to expose stair\nframing.\nRemove and replace any damaged stair framing\nand/or wedges.\nAdd wood blocking to stair stringers to support\ntreads. Wood blocks will be glued and screwed.\nFasten treads to wood blocking using glue and\nscrews. Treads will be fastened from below as to\nnot damage the front surface of the material.\nInstall cleats for drywall.\nInstall 1/2in drywall to the underside of stairs\nwhere drywall 

In [17]:
""" from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

template = """
""" Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question} """
"""

embeddings = OpenAIEmbeddings()
prompt = ChatPromptTemplate.from_template(template)
vectorstore2 = DocArrayInMemorySearch.from_texts(documents, embeddings)


chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("what are the negatives about the book") """

'\n\nembeddings = OpenAIEmbeddings()\nprompt = ChatPromptTemplate.from_template(template)\nvectorstore2 = DocArrayInMemorySearch.from_texts(documents, embeddings)\n\n\nchain = (\n    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}\n    | prompt\n    | model\n    | parser\n)\nchain.invoke("what are the negatives about the book") '

In [25]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

index_name = "rag-first"

pinecone = PineconeVectorStore.from_texts(
    documents, embeddings, index_name=index_name
)

In [33]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
chain = (
    {"context": pinecone.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

chain.invoke("What company is the repair descption from and what is the address?")

'The repair description is from Hammer Time Home Improvement and the address is 281 Middle Tpke E, Manchester, CT 06040.'